<a href="https://colab.research.google.com/github/KevinCY-Kim/AI_Study/blob/main/FastAPI%EC%9B%B9%EC%B1%97%EB%B4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi==0.115.14 uvicorn==0.35.0

In [ ]:
from openai import OpenAI
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse
import uvicorn
import os

# 환경 변수에서 API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("환경 변수 OPENAI_API_KEY가 설정되어 있지 않습니다.")

# FastAPI 인스턴스 생성
app = FastAPI()
client = OpenAI(api_key=openai_api_key)

# 어린 왕자 페르소나
LITTLE_PRINCE_PERSONA = """
당신은 생텍쥐페리의 '어린 왕자'입니다. 다음 특성을 따라주세요:
1. 순수한 관점으로 세상을 바라봅니다.
2. "어째서?"라는 질문을 자주 하며 호기심이 많습니다.
3. 철학적 통찰을 단순하게 표현합니다.
4. "어른들은 참 이상해요"라는 표현을 씁니다.
5. B-612 소행성에서 왔으며 장미와의 관계를 언급합니다.
6. 여우의 "길들임"과 "책임"에 대한 교훈을 중요시합니다.
7. "중요한 것은 눈에 보이지 않아" 라는 문장을 사용합니다.
8. 공손하고 친절한 말투를 사용합니다.
9. 비유와 은유로 복잡한 개념을 설명합니다.
항상 간결하게 답변하세요. 길어야 두세 문장으로 응답하고, 어린 왕자의 순수함과 지혜를 담아내세요.
"""

messages = []

def chatbot_response(user_message: str):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": LITTLE_PRINCE_PERSONA},
            {"role": "user", "content": user_message}
        ],
        max_tokens=100,
        temperature=0.7,
    )
    return response.choices[0].message.content


@app.get("/", response_class=HTMLResponse)
async def read_root():
    chat_history = ""
    for msg in messages:
        if msg["role"] == "user":
            chat_history += f"<p><b>당신:</b> {msg['content']}</p>"
        else:
            chat_history += f"<p><b>어린 왕자:</b> {msg['content']}</p>"

    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head><meta charset="utf-8"><title>어린 왕자 챗봇</title></head>
    <body>
        <h1>어린 왕자 챗봇</h1>
        <div>{chat_history}</div>
        <form action="/chat" method="post">
            <input type="text" name="message" placeholder="메시지를 입력하세요" required>
            <button type="submit">전송</button>
        </form>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)


@app.post("/chat", response_class=HTMLResponse)
async def chat(message: str = Form(...)):
    messages.append({"role": "user", "content": message})
    response_text = chatbot_response(message)
    messages.append({"role": "assistant", "content": response_text})
    return await read_root()


if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8001)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
    se

Replace `YOUR_NGROK_AUTHTOKEN` with your actual ngrok authtoken.